<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYVERSION_OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [ ]:
%pip install llama-index-readers-wikipedia -q

In [ ]:
!pip install llama-index -q

In [3]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

In [ ]:
!pip install wikipedia -q

In [5]:
reader = WikipediaReader()

In [6]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

In [ ]:
!pip install langchain -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q
%pip install colab-env --upgrade --quiet --root-user-action=ignore -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

In [10]:
response=tool(["Montreal"], query_str="where is located the Montreal city?")

In [11]:
response.content

'Montreal is located in the southwest of the province of Quebec, Canada. It is situated on the Island of Montreal at the confluence of the Saint Lawrence and Ottawa Rivers.'

In [12]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [49]:
from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain_openai import ChatOpenAI, OpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)

#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)
llm = ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)

tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

agent = initialize_agent(
    [lc_tool],
    #tools,
    llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

In [50]:
agent_chain.run("What's my friend Rafael's surname?") ## GPT-3.5
# Answer with 'Reinaldo'



Can you tell me Rafael's surname?
Reinaldo


"Rafael's surname is Reinaldo."

In [15]:
#agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"},query_str="What's the arts and culture scene in montreal?")

agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"})



Observation: Montreal boasts a vibrant arts and culture scene, earning it the title of "Canada's Cultural Capital." The city is known for its diverse cultural communities, which have contributed to a unique local culture. Montreal hosts numerous cultural events and festivals throughout the year, including the Montreal International Jazz Festival, the largest jazz festival globally, and the Just for Laughs comedy festival, the largest of its kind. The city's downtown area comes alive during the summer with a variety of cultural and social events. Montreal is also recognized for its classical arts scene, with the Place des Arts serving as a hub for various performances, including those by the Montreal Symphony Orchestra and the Opéra de Montréal. Additionally, Montreal's vibrant nightlife is considered an integral part of its cultural ecosystem.
Thought:

"Montreal boasts a vibrant arts and culture scene, earning it the title of 'Canada's Cultural Capital.' The city is known for its diverse cultural communities, which have contributed to a unique local culture. Montreal hosts numerous cultural events and festivals throughout the year, including the Montreal International Jazz Festival, the largest jazz festival globally, and the Just for Laughs comedy festival, the largest of its kind. The city's downtown area comes alive during the summer with a variety of cultural and social events. Montreal is also recognized for its classical arts scene, with the Place des Arts serving as a hub for various performances, including those by the Montreal Symphony Orchestra and the Opéra de Montréal. Additionally, Montreal's vibrant nightlife is considered an integral part of its cultural ecosystem."

In [ ]:
response=agent.run(input={"pages": "Montreal", "query_str": "What is the best restaurant in Montreal?"})

In [46]:
response

"Some of the best restaurants in Montreal are Joe Beef, Au Pied de Cochon, Toqué!, Schwartz's Deli, L'Express, and Liverpool House."

In [18]:
!pip install llama_index -q
!pip install langchain -q
!pip install wikipedia -q

In [31]:
import os
import openai

import llama_index

#from llama_index.tools.ondemand_loader_tool import OnDemandLoaderTool
## added by frank morales
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool

from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

reader = WikipediaReader()

tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

# run only the llama_Index Tool by itself
#tool(["Cuba"], query_str="What's the arts and culture scene in Cuba?")

# run tool from LangChain Agent
lc_tool = tool.to_langchain_structured_tool(verbose=True)
#response=lc_tool.run(tool_input={"pages": ["Cuba"], "query_str": "What's the capital city of Cuba?"})
response=lc_tool.run(tool_input={"pages": ["Cuba"], "query_str": "What's the arts and culture scene in Cuba?"})

Cuba has a rich arts and culture scene with influences from various sources. The country is known for its vibrant music scene, including genres like son, salsa, and Afro-Cuban jazz. Cuban visual arts have a strong tradition, with artists like Wifredo Lam and Amelia Peláez gaining international recognition. Cuban literature has produced notable figures such as José Martí and Alejo Carpentier. The country also has a thriving dance scene, with the famous Cuban ballet being a significant cultural export. Overall, Cuba's arts and culture scene is diverse and dynamic, reflecting the country's unique history and heritage.

Initialise LangChain Agent

In [ ]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False
)

#agent.run("Tell me about the arts and culture of Berlin")

response=agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"}) ## GPT-3.5

In [30]:
response

"Montreal boasts a vibrant arts and culture scene, earning it the title of 'Canada's Cultural Capital.' The city is known for its diverse cultural communities, which have contributed to a unique local culture. Montreal hosts numerous cultural events and festivals throughout the year, including the Montreal International Jazz Festival, Just for Laughs comedy festival, Montreal World Film Festival, and many more. The city's downtown area comes alive during the summer with a variety of cultural and social activities. Montreal is also recognized for its classical arts scene, with venues like Place des Arts hosting performances by renowned orchestras, opera companies, ballet troupes, and avant-garde dance groups. Additionally, Montreal's architecture reflects its rich history, with a mix of colonial buildings in Old Montreal and iconic modern structures like the Olympic Stadium and Habitat 67."